In [1]:
import pandas as pd
import re 
from fractions import Fraction

In [2]:
df = pd.read_csv('products_2.csv')

In [3]:
df.head()

,id,product_name,product_category_id,product_category_name,product_type_numeric,product_type_text,product_manufacturer_id,product_manufacturer_name
0,512842,Layers Mash,318,Feeds,1.0,kg,608.0,Lucky Feeds
1,513291,Layers Mash,313,Minerals and Supplements,70.0,kg,2314.0,Mapato Feeds
2,514788,Layers Mash,318,Feeds,10.0,kg,5311.0,murimi feeds
3,514789,Layers Mash,318,Feeds,1.0,kg,5311.0,murimi feeds
4,514845,Layers Mash,318,Feeds,5.0,kg,25878.0,Meru Central Ltd


In [4]:
df.describe()

,id,product_category_id,product_type_numeric,product_manufacturer_id
count,50000.000000,50000.000000,4.679300e+04,49601.000000
mean,303101.912560,396.820060,3.276169e+02,5128.425616
std,169259.928945,593.278557,5.707254e+04,6982.752270
min,27.000000,1.000000,0.000000e+00,0.000000
25%,161488.750000,313.000000,2.000000e+00,136.000000
50%,295918.000000,318.000000,1.000000e+01,2676.000000
75%,458799.750000,318.000000,5.000000e+01,6629.000000
max,573441.000000,9609.000000,1.234568e+07,28446.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         50000 non-null  int64  
 1   product_name               50000 non-null  object 
 2   product_category_id        50000 non-null  int64  
 3   product_category_name      50000 non-null  object 
 4   product_type_numeric       46793 non-null  float64
 5   product_type_text          47514 non-null  object 
 6   product_manufacturer_id    49601 non-null  float64
 7   product_manufacturer_name  49575 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 3.1+ MB


In [6]:
# Checking count of unique values in the categorical column

categorical_columns = df.select_dtypes(include=['object']).columns

# create a data frame to store the count of unique values in the categorical columns

unique_values = pd.DataFrame(columns=['column_name', 'unique_values_count'])

for column in categorical_columns:
    unique_values = unique_values._append({'column_name': column, 'unique_values_count': df[column].nunique()}, ignore_index=True)

unique_values




,column_name,unique_values_count
0,product_name,13576
1,product_category_name,115
2,product_type_text,579
3,product_manufacturer_name,4414


In [7]:
# Checking for missing values

missing_values = pd.DataFrame(columns=['column_name', 'missing_values_count'])

for column in df.columns:
    missing_values = missing_values._append({'column_name': column, 'missing_values_count': df[column].isnull().sum()}, ignore_index=True)
    
missing_values

,column_name,missing_values_count
0,id,0
1,product_name,0
2,product_category_id,0
3,product_category_name,0
4,product_type_numeric,3207
5,product_type_text,2486
6,product_manufacturer_id,399
7,product_manufacturer_name,425


# Data Cleaning

In [8]:
# Putting categorical text into Camel Case for uniformity

df['product_name'] = df['product_name'].str.title()

df['product_manufacturer_name'] = df['product_manufacturer_name'].str.title()

df['product_category_name'] = df['product_category_name'].str.title()

In [9]:
def clean_text(text):
    # Replace hyphens and remove brackets
    text = text.replace('-', ' ')
    text = text.replace('_', ' ')
    text = text.replace('(', '').replace(')', ' ')
    text = text.replace('[', '').replace(']', ' ')
    return text

df['product_name'] = df['product_name'].apply(clean_text)

In [10]:
# Extracting the numeric and unit if missing from the product name

def extract_and_fill(row):
    pattern = r'(\d+)\s*(kg|g|l|ml|Litre|Kilogram|Kg|G|kg|KG|ML|Ml|Kgs)'
    match = re.search(pattern, row['product_name'], re.IGNORECASE)
    if match:
        numeric, unit = match.groups()
        if pd.isnull(row['product_type_numeric']):
            row['product_type_numeric'] = int(numeric)
        if pd.isnull(row['product_type_text']):
            row['product_type_text'] = unit.lower()  # Convert to lowercase for consistency
    return row


# Apply the function to the dataframe
df = df.apply(extract_and_fill, axis=1)
 

In [11]:
# Function to remove numeric and unit from product_name
def remove_numeric_and_unit(row):
    pattern = r'(\d+)\s*(kg|g|l|ml|Litre|Kilogram|Kg|G|kg|KG|ML|Ml|Kgs)'
    row['product_name'] = re.sub(pattern, '', row['product_name'], flags=re.IGNORECASE).strip()
    return row

# Apply the function to remove numeric values and units from product_name
df = df.apply(remove_numeric_and_unit, axis=1)



In [12]:
missing_values = pd.DataFrame(columns=['column_name', 'missing_values_count'])

for column in df.columns:
    missing_values = missing_values._append({'column_name': column, 'missing_values_count': df[column].isnull().sum()}, ignore_index=True)
    
missing_values

,column_name,missing_values_count
0,id,0
1,product_name,0
2,product_category_id,0
3,product_category_name,0
4,product_type_numeric,2930
5,product_type_text,2256
6,product_manufacturer_id,399
7,product_manufacturer_name,425


In [13]:
# Cleaning up the product_type_text column

df['product_type_text'] = df['product_type_text'].str.lower()

df['product_type_text'] = df['product_type_text'].str.strip()


    

In [14]:
product_type_text = df['product_type_text'].value_counts()


In [15]:
# Extracting numericals and fractions from product type text

def extract_numerical(row):
    pattern = r'(\d+/\d+|\d+)'
    match = re.search(pattern, str(row['product_type_text']), re.IGNORECASE)
    if match:
        numeric = match.group()
        row['product_type_numeric'] = float(Fraction(numeric))
    return row

# Apply the function to the dataframe

df = df.apply(extract_numerical, axis=1)




In [16]:
missing_values = pd.DataFrame(columns=['column_name', 'missing_values_count'])

for column in df.columns:
    missing_values = missing_values._append({'column_name': column, 'missing_values_count': df[column].isnull().sum()}, ignore_index=True)
    
missing_values

,column_name,missing_values_count
0,id,0
1,product_name,0
2,product_category_id,0
3,product_category_name,0
4,product_type_numeric,2355
5,product_type_text,2256
6,product_manufacturer_id,399
7,product_manufacturer_name,425


In [17]:


# Function to extract units
def extract_unit(row):
    pattern = r'(kilogram|gorogoro|litre|sack|kg|gms|ml|l|g|bag)'
    match = re.search(pattern, str(row['product_type_text']), re.IGNORECASE)
    if match:
        unit = match.group()
        row['product_type_text'] = unit
    return row

# Apply the function to the dataframe

df = df.apply(extract_unit, axis=1)

In [18]:
# replace dictionary for product_type_text column

replace_dict = {
    'kg': 'kg',
    'g': 'g',
    'l': 'l',
    'ml': 'ml',
    'litre': 'l',
    'kilogram':'kg',
    'kgs': 'kg',
    'ltr': 'l',
    'gms': 'g',
    'mls': 'ml',
    'lt': 'l',
    'ltrs': 'l',
    'grms': 'g',
    'kg.': 'kg',
    'kgs.': 'kg',
    'gms': 'g',
    'gm': 'g',
    'grms': 'g',
    'kg bags': 'kg',
    'ml.': 'ml',
    'kilos': 'kg',
    'grams': 'g',
    'litres': 'l',
    'lr': 'l',
    'kilo': 'kg',
    'lts': 'l',
    'kgd': 'kg',
    'okg': 'kg',
    'kilograms': 'kg',
    'gr': 'g',
    'korokoro': 'gorogoro',
    'ks': 'kg',
    'kf': 'kg',
    'feeds':'feeds',
    'unit': 'unit',
    'units': 'unit',
    'kh': 'kg',
    'gorogoro': 'gorogoro',
    'sack': 'sack',
    'bag': 'bag',
    'bags': 'bag',
    'sacks': 'sack',

}



In [19]:
# Applying the dictionary if value isn't found its replaced by other

def replace_with_dic(value, replace_dict):
    return replace_dict.get(value, 'other')


df['product_type_text'] = df['product_type_text'].apply(replace_with_dic, args=(replace_dict,))
    

In [20]:
missing_values = pd.DataFrame(columns=['column_name', 'missing_values_count'])

for column in df.columns:
    missing_values = missing_values._append({'column_name': column, 'missing_values_count': df[column].isnull().sum()}, ignore_index=True)
    
missing_values

,column_name,missing_values_count
0,id,0
1,product_name,0
2,product_category_id,0
3,product_category_name,0
4,product_type_numeric,2355
5,product_type_text,0
6,product_manufacturer_id,399
7,product_manufacturer_name,425


In [21]:
# Check the value counts of the product_type_text column

product_type_text = df['product_type_text'].value_counts()